In [2]:
import numpy as np


In [45]:
# dataset generation
# points
A = np.matrix([[10.0,10.0,10.0],
               [20.0,10.0,10.0],
               [20.0,10.0,15.0],])

B = np.matrix([[18.8106,17.6222,12.8169],
               [28.6581,19.3591,12.8173],
               [28.9554, 17.6748, 17.5159],])
n = B.shape[0]

# transformations
T_src = np.matrix([[0.9848, 0.1737,0.0000,-11.5865],
                   [-0.1632,0.9254,0.3420, -7.621],
                   [0.0594,-0.3369,0.9400,2.7752],
                   [0.0000, 0.0000,0.0000,1.0000]])
T_dst = np.matrix([[0.9848, 0.1737,0.0000,-11.5859],
                   [-0.1632,0.9254,0.3420, -7.621],
                   [0.0594,-0.3369,0.9400,2.7755],
                   [0.0000, 0.0000,0.0000,1.0000]])

scaling = True

In [46]:
# https://gist.github.com/oshea00/dfb7d657feca009bf4d095d4cb8ea4be
def rigid_transform_3D(A, B, scale):

    assert len(A) == len(B)
    N = A.shape[0]

    # mean of each point cloud
    Am = np.mean(A, axis=0)
    Bm = np.mean(B, axis=0)

    # centered point clouds
    Ac = A - np.tile(Am, (N, 1))
    Bc = B - np.tile(Bm, (N, 1))

    H = np.transpose(Bc) * Ac # NOTE: Kabsch; H = P^T \cdot Q
    if scale: H /= N

    """
    Based on rotation formula, optimal rotation R is

    R = sqrt(H^T \cdot H) \cdot H^-1

    since directly solving this is complicated and hard to handle edge cases (e.g., singular H), use SVD
    """

    U, S, Vt = np.linalg.svd(H) # NOTE: Kabsch; H = USV^T
    
    R = Vt.T * U.T # NOTE: Kabsch; R = V \cdot U^T

    # special reflection case
    if np.linalg.det(R) < 0:
        print("[DEBUG] Reflection detected")
        Vt[2, :] *= -1
        R = Vt.T * U.T
    
    if scale:
        Avar = np.var(A, axis=0).sum()
        c = 1 / (1 / Avar * np.sum(S)) # scale singular value
        t = -R * (Bm.T * c) + Am.T
        
    else:
        c = 1
        t = -R.dot(Bm.T) + Am.T

    return c, R, t

s, R, t = rigid_transform_3D(A, B, True)
print(s)
print(f"{R=}")
print(f"{t=}")

1.0000149841190074
R=matrix([[ 9.84798975e-01,  1.73697950e-01,  4.18639469e-05],
        [-1.63234699e-01,  9.25394020e-01,  3.42053127e-01],
        [ 5.93751863e-02, -3.36860402e-01,  9.39680614e-01]])
t=matrix([[-11.5863681 ],
        [ -7.62126063],
        [  2.77535618]])


In [48]:
# validation

A2 = (R * B.T) + np.tile(t, (1, n))
A2 = A2.T
print(A)
print(A2)



[[10. 10. 10.]
 [20. 10. 10.]
 [20. 10. 15.]]
[[ 9.99976807  9.99973596  9.99981013]
 [19.99927196  9.99973596  9.99979032]
 [19.99968994  9.99973596 14.99999987]]


In [49]:
# evaluate with 
P_selected = np.asmatrix( np.loadtxt("P_selected.txt"))
Q_selected = np.asmatrix(np.loadtxt("Q_selected.txt"))

print(P_selected)
print(Q_selected)
print(type(P_selected))

[[2.16015625 1.72265625 1.86595058]
 [2.17357731 1.60546875 1.85546875]
 [2.69921875 2.25390625 1.07963181]]
[[2.04720283 2.00390625 1.62109375]
 [1.72302353 1.94140625 1.40234375]
 [1.23046875 2.46200871 1.55078125]]
<class 'numpy.matrix'>


In [51]:
# without scaling

s, R, t = rigid_transform_3D(P_selected, Q_selected, False)
P_selected2 = (R * Q_selected.T) + np.tile(t, (1, 3))
P_selected2 = P_selected2.T
print(P_selected)
print(P_selected2)

[[2.16015625 1.72265625 1.86595058]
 [2.17357731 1.60546875 1.85546875]
 [2.69921875 2.25390625 1.07963181]]
[[2.105586   1.81504416 1.93535721]
 [2.31675078 1.6321899  1.65460148]
 [2.61061553 2.13479719 1.21109245]]


In [52]:
# with scaling

s, R, t = rigid_transform_3D(P_selected, Q_selected, True)
P_selected2 = (R * Q_selected.T) + np.tile(t, (1, 3))
P_selected2 = P_selected2.T
print(P_selected)
print(P_selected2)

[DEBUG] Reflection detected
[[2.16015625 1.72265625 1.86595058]
 [2.17357731 1.60546875 1.85546875]
 [2.69921875 2.25390625 1.07963181]]
[[2.23071229 0.90547425 1.38836503]
 [2.44187707 0.72261999 1.10760929]
 [2.73574182 1.22522728 0.66410027]]
